# Import libraries

In [1]:
import pandas as pd
import numpy as np
import re

# Import CSV Data - Oddle_Assignment Orders Data.csv

In [2]:
# Create pandas DF
orders_df = pd.read_csv("Oddle_Assignment Orders Data.csv")

In [3]:
# Check imported dataframe
orders_df.head()

,Unnamed: 0,submitted_on,pickup,delivery_date,customer_address_postal,promotion_code,lead_time,basket_size,customer_index,menu_id,merchant_type,organisation_name_provided
0,1,2016-12-19 03:55:08,0,2016-12-21 12:00:00,823663,NaN,2160.0,64.45,27874,1,Cakes,0.0
1,2,2016-12-17 12:25:17,0,2016-12-17 14:00:00,391010,NaN,60.0,74.99,7983,2,Korean Chicken,0.0
2,3,2016-12-19 15:31:31,1,2016-12-24 04:00:00,NaN,NaN,2880.0,77.66,21225,3,Cakes,0.0
3,4,2016-12-18 17:38:36,1,2016-12-26 04:30:00,575312,NaN,4320.0,57.27,42870,4,Cakes,0.0
4,5,2016-12-18 03:37:55,0,2016-12-23 09:15:00,188770,"1212,",120.0,138.95,11021,5,Dim Sum,0.0


In [4]:
## Reset the Indexes
orders_df = orders_df.drop('Unnamed: 0', axis=1)

In [5]:
orders_df.head()

,submitted_on,pickup,delivery_date,customer_address_postal,promotion_code,lead_time,basket_size,customer_index,menu_id,merchant_type,organisation_name_provided
0,2016-12-19 03:55:08,0,2016-12-21 12:00:00,823663,NaN,2160.0,64.45,27874,1,Cakes,0.0
1,2016-12-17 12:25:17,0,2016-12-17 14:00:00,391010,NaN,60.0,74.99,7983,2,Korean Chicken,0.0
2,2016-12-19 15:31:31,1,2016-12-24 04:00:00,NaN,NaN,2880.0,77.66,21225,3,Cakes,0.0
3,2016-12-18 17:38:36,1,2016-12-26 04:30:00,575312,NaN,4320.0,57.27,42870,4,Cakes,0.0
4,2016-12-18 03:37:55,0,2016-12-23 09:15:00,188770,"1212,",120.0,138.95,11021,5,Dim Sum,0.0


# Dataset Overview

## Check Dimensions of the Dataset.

In [6]:
# Check Dimensions
orders_df.shape

# 22121 observations (rows)
# 11 Features (cols)

(22121, 11)

## Check for Missing Values

In [7]:
# Check for Missing Values
orders_df.isnull().sum()

## customer_address_postal has 3371 nulls
## promotion_code has 19237 nulls
## organisation_Name_provided has 7 nulls

submitted_on                      0
pickup                            0
delivery_date                     0
customer_address_postal        3371
promotion_code                19237
lead_time                         1
basket_size                       0
customer_index                    0
menu_id                           0
merchant_type                     0
organisation_name_provided        7
dtype: int64

## Check for Datatypes (changed accordingly for work to be done in python)

In [8]:
#Check data types
orders_df.dtypes

## From the data dictionary provided in the Assignment PDF
# submitted_on == Date Time for order submission
# pickup == 1/0 binary
# delivery_date == Date Time
# customer_address_postal   == String -> need to convert to 6 digit integer
# promotion_code == String obj
# lead_time == integer (No. of mins) -> Might convert to time object
# basket_size  == float ($$ worth of basket)
# customer_index == customer ID, unique integer (Match from customer table)
# menu_id == Merchant ID
# merchant_type == String obj categorical
# organisation_name_provided == binary 1/0 (Corporate or not)

submitted_on                   object
pickup                          int64
delivery_date                  object
customer_address_postal        object
promotion_code                 object
lead_time                     float64
basket_size                   float64
customer_index                  int64
menu_id                         int64
merchant_type                  object
organisation_name_provided    float64
dtype: object

## Check for Duplicates, Validity etc

### submitted_on

In [9]:
# Check duplicates for 'submitted_on'
orders_df['submitted_on'].duplicated().sum()

# 6 duplicates of the created date and time

### However this is to be ignored. Nothing wrong with users ordering simultaneously.

6

In [10]:
# Adjust datatypes for the dataframe:
## submitted_on = datetime.
orders_df['submitted_on'] = pd.to_datetime(orders_df['submitted_on'])

In [11]:
orders_df['submitted_on'].max()

Timestamp('2018-09-30 22:56:17')

In [12]:
orders_df['submitted_on'].min()

Timestamp('2016-10-01 03:56:03')

### pickup

In [13]:
## Check PickUp
orders_df['pickup'].value_counts()

# No funny values - Just binary 1/0

0    16167
1     5954
Name: pickup, dtype: int64

### delivery_date (Date time)

In [14]:
# Check duplicates for 'delivery_date'
orders_df['delivery_date'].duplicated().sum()

# 11643 duplicates of the created date and time

### However this is to be ignored. Nothing wrong with users ordering simultaneously.

11643

In [15]:
# Adjust datatypes for the dataframe:
## delivery_date = datetime.
orders_df['delivery_date'] = pd.to_datetime(orders_df['delivery_date'])

In [16]:
orders_df['delivery_date'].max()

Timestamp('2018-12-30 06:00:00')

In [17]:
orders_df['delivery_date'].min()

Timestamp('2016-10-01 07:00:00')

### customer_address_postal

In [18]:
# Check validity/errors for 'customer_address_postal'
# Not concerned about duplicates because can be delivered to the same place

orders_df['customer_address_postal'].isnull().sum()

3371

In [19]:
## Postalcode checker function - Check that the postal code entered by the user is in a valid 6 digits
## returns a boolean to a result column
## We will approach the clean up or the replacement etc after we do the validity check.
## Potential errors that we are looking for are things like S123456 (Because we are used to putting a prefix 'S')
## Additional chars other than the 6 digits
## Will do format conversion after we check the validity

def is_valid_postal(code):
    ## Current format is still a string
    if type(code) == str:
        if len(code) == 6:
            return True
        else:
            return False
    else:
        return False

In [20]:
## Testing.. Ok
is_valid_postal(123123)

False

In [21]:
## Apply the function
orders_df['postal_valid'] = orders_df['customer_address_postal'].apply(lambda x: is_valid_postal(x))

In [22]:
## Check the counts
orders_df['postal_valid'].value_counts()

True     18449
False     3672
Name: postal_valid, dtype: int64

In [23]:
orders_df.loc[(orders_df['postal_valid'] == False) &  (orders_df['customer_address_postal'].notnull())]

## We see 'Singapore, Blocks, S, we still see 6 digits. But there may be a SPACE or something in there.

,submitted_on,pickup,delivery_date,customer_address_postal,promotion_code,lead_time,basket_size,customer_index,menu_id,merchant_type,organisation_name_provided,postal_valid
26,2016-12-19 12:36:34,0,2016-12-23 02:45:00,Singapore 319400,NaN,120.0,173.08,11045,5,Dim Sum,0.0,False
50,2016-12-28 03:15:37,0,2016-12-30 07:00:00,Block 264,NaN,2160.0,85.89,28027,1,Cakes,0.0,False
121,2016-12-20 15:57:36,1,2016-12-24 04:00:00,S570442,NaN,2880.0,51.95,21236,3,Cakes,0.0,False
133,2016-12-21 04:29:31,1,2016-12-24 04:00:00,Singapore 570308,NaN,2160.0,162.93,27939,1,Cakes,0.0,False
178,2016-12-22 22:26:02,0,2016-12-28 09:30:00,1543,NaN,2880.0,46.39,21248,3,Cakes,0.0,False
345,2016-12-30 01:44:04,0,2016-12-30 07:00:00,71 Ayer Raj,NaN,60.0,63.28,630,2,Korean Chicken,0.0,False
446,2017-01-03 12:12:15,0,2017-01-04 07:00:00,567720,NaN,150.0,88.87,19605,5,Dim Sum,0.0,False
575,2017-12-04 04:31:51,0,2017-12-16 02:30:00,530911,NaN,2880.0,82.45,21353,3,Cakes,0.0,False
593,2017-12-04 09:55:52,0,2017-12-05 05:30:00,SINGAPORE 187969,NaN,150.0,181.44,11255,5,Dim Sum,1.0,False
664,2017-12-05 15:12:20,0,2017-12-17 03:30:00,"Marina Bay Residences, 018980",NaN,150.0,251.76,13444,5,Dim Sum,0.0,False


In [24]:
## Create Cleaned up / Corrected postal codes
## We only want to extract 6 digits intact -> replace the problem ones with NULL

def clean_postals(code):
    code = str(code)
    x = re.findall(r"\D(\d{6})\D", " "+code+" ")
    if len(x) == 0:
        #print('Not Valid')
        return np.nan
    else:
       #print(x[0])
        return x[0]

In [25]:
orders_df['postal_good'] = orders_df['customer_address_postal'].apply(lambda x: clean_postals(x))

s= "123108"
x = re.findall(r"\D(\d{6})\D", " "+s+" ")

if len(x) == 0:
    print('Not Valid')
    return np.nan
else:
    print(x[0])
    return x[0]

In [26]:
orders_df['postal_good'].isnull().sum()

3527

In [27]:
## Managed to correct these postal codes
orders_df.loc[orders_df['postal_good'].notnull() & (orders_df['postal_valid']==False)][['customer_address_postal','postal_good']]

,customer_address_postal,postal_good
26,Singapore 319400,319400
121,S570442,570442
133,Singapore 570308,570308
446,567720,567720
575,530911,530911
593,SINGAPORE 187969,187969
664,"Marina Bay Residences, 018980",018980
703,S257889,257889
715,238843,238843
739,Singapore 079908,079908


<<Use the **postal_good** column >>>>

### promotion_code

In [28]:
# Check types for 'promotion code'
orders_df['promotion_code'].value_counts()
#orders_df['promotion_code'].count()
## 2884 codes
## Clean out the ,
## Convert to UPPERCASE

FIRST10,            1093
FRIENDS,             437
FIRST5               428
FIRST5,              360
10OFF,               151
1212,                 53
GIVEMESALAD,          52
UglyCakeKakis,        50
10OFF                 48
WATG,                 26
YayDads               19
WELCOMEBACK,          12
5OFF50                11
4FLASH,               11
FT12                  10
10                     9
FlavourOfTheWeek       7
HEALTHY10,             6
20YCE,                 6
AACAKELOVE,            6
HUNGRY4SC              6
XMAS20,                5
singtel,               5
LIGHTBITES15,          5
SG52,                  5
CORPORATE,             5
GSS3,                  4
5off50                 4
15,                    3
Hungry4sc              3
                    ... 
DimSumParty50,         1
FF20,                  1
cv20,                  1
APRILGIVEAWAY,         1
TanTanFriends          1
SITJUNE2018            1
xoddle10,              1
,FIRST10               1
hungry4sc              1


In [29]:
## Function to remove , and to convert all to UPPERCASE

def clean_promo(code):
    if type(code) == str:
        code = code.upper()
        code = code.replace(',','') ##remove ','
        return code
    else:
        return code ## Keep numeric codes and nulls

In [30]:
## Apply the function
orders_df['promo_good'] = orders_df['promotion_code'].apply(lambda x: clean_promo(x))

In [31]:
orders_df['promo_good'].value_counts()  ##Corrected the grouping counts

FIRST10               1094
FIRST5                 788
FRIENDS                437
10OFF                  200
1212                    53
GIVEMESALAD             52
UGLYCAKEKAKIS           50
WATG                    26
YAYDADS                 19
5OFF50                  15
WELCOMEBACK             13
4FLASH                  11
HUNGRY4SC               10
FT12                    10
10                       9
FLAVOUROFTHEWEEK         7
HEALTHY10                7
20YCE                    6
AACAKELOVE               6
CORPORATE                5
LIGHTBITES15             5
SG52                     5
SINGTEL                  5
XMAS20                   5
15                       5
GSS3                     4
CV20                     3
FT10                     3
SINGAPORELICIOUZ10       2
HEALTHYFOOD              2
SITJUNE2018              1
69JK71                   1
THANKYOULYNSS            1
HAPPYBIRTHDAYCROWN       1
DND604                   1
DND852                   1
A67DBG                   1
V

In [32]:
orders_df['promo_good'].count()

2884

### lead_time

In [33]:
orders_df['lead_time']

0        2160.0
1          60.0
2        2880.0
3        4320.0
4         120.0
5          60.0
6         120.0
7        2880.0
8         120.0
9          60.0
10         60.0
11         60.0
12        120.0
13        120.0
14        120.0
15        120.0
16        120.0
17        120.0
18       2880.0
19        120.0
20         60.0
21         60.0
22       2160.0
23       4320.0
24        150.0
25       4320.0
26        120.0
27        150.0
28        120.0
29        120.0
          ...  
22091      90.0
22092     720.0
22093    2880.0
22094    2880.0
22095    4320.0
22096    4320.0
22097     720.0
22098     150.0
22099     150.0
22100     150.0
22101     720.0
22102      15.0
22103    4320.0
22104     720.0
22105      20.0
22106     150.0
22107     720.0
22108      90.0
22109    4320.0
22110      90.0
22111     120.0
22112     120.0
22113    4320.0
22114      90.0
22115     720.0
22116     150.0
22117      30.0
22118     150.0
22119      30.0
22120      90.0
Name: lead_time, Length:

In [34]:
orders_df['lead_time'].max()

4320.0

In [35]:
orders_df['lead_time'].min()

0.0

In [36]:
## Check orders with NO LEAD TIME
orders_df.loc[orders_df['lead_time']==0]

## Merchants that dont have lead time, 1 Cake and the Salad

,submitted_on,pickup,delivery_date,customer_address_postal,promotion_code,lead_time,basket_size,customer_index,menu_id,merchant_type,organisation_name_provided,postal_valid,postal_good,promo_good
474,2017-01-04 09:15:06,0,2017-01-07 09:30:00,NaN,NaN,0.0,98.63,21300,3,Cakes,0.0,False,NaN,NaN
569,2017-12-08 02:27:39,0,2017-12-08 03:30:00,349327,NaN,0.0,16.10,105,7,Salad,1.0,True,349327,NaN
618,2017-12-14 05:36:26,0,2017-12-15 03:30:00,537055,NaN,0.0,22.28,201,7,Salad,0.0,True,537055,NaN
707,2018-02-02 00:46:13,0,2018-02-02 03:30:00,408931,NaN,0.0,18.21,372,7,Salad,0.0,True,408931,NaN
712,2018-02-02 02:18:34,0,2018-02-02 03:30:00,408734,NaN,0.0,10.84,8,7,Salad,1.0,True,408734,NaN
781,2018-02-05 21:24:52,0,2018-02-06 03:30:00,408734,NaN,0.0,14.28,8,7,Salad,1.0,True,408734,NaN
783,2018-02-07 02:06:59,0,2018-02-07 03:30:00,408538,NaN,0.0,13.51,183,7,Salad,0.0,True,408538,NaN
829,2018-02-06 12:59:21,0,2018-02-07 03:30:00,408931,NaN,0.0,13.70,372,7,Salad,0.0,True,408931,NaN
841,2018-02-07 11:57:17,0,2018-02-08 03:30:00,408931,NaN,0.0,16.64,372,7,Salad,0.0,True,408931,NaN
846,2018-02-12 09:54:54,0,2018-02-13 03:30:00,408538,NaN,0.0,29.81,6,7,Salad,0.0,True,408538,NaN


In [37]:
orders_df['lead_time'].isnull().sum()

# 1 Null

1

In [38]:
orders_df.loc[orders_df['lead_time'].isnull()]

,submitted_on,pickup,delivery_date,customer_address_postal,promotion_code,lead_time,basket_size,customer_index,menu_id,merchant_type,organisation_name_provided,postal_valid,postal_good,promo_good
12821,2018-04-25 12:22:00,1,2018-04-30 09:00:00,NaN,NaN,NaN,123.73,46748,4,Cakes,0.0,False,NaN,NaN


In [39]:
orders_df.loc[orders_df['lead_time'].isnull()]['delivery_date'] - orders_df.loc[orders_df['lead_time'].isnull()]['submitted_on']

12821   4 days 20:38:00
dtype: timedelta64[ns]

In [40]:
## to be safe we impute with the Max lead time of 3 days. (4320 mins)
x = orders_df[orders_df['lead_time'].isnull()]

orders_df.loc[x.index, 'lead_time'] = 4320.0

In [41]:
orders_df.loc[orders_df['lead_time'].isnull()] ##Check for replacement done.

,submitted_on,pickup,delivery_date,customer_address_postal,promotion_code,lead_time,basket_size,customer_index,menu_id,merchant_type,organisation_name_provided,postal_valid,postal_good,promo_good


In [42]:
## Function to convert the minutes into days/hrs/min for easier viewing

def fix_time(time):
    hrs, mins = divmod(time, 60)
    days, hrs = divmod(hrs, 24)
    #print ("%d:%02d:%02d" % (days, hrs, mins))
    return ("%d:%02d:%02d" % (days, hrs, mins))

In [43]:
orders_df['lead_time_good'] = orders_df['lead_time'].apply(lambda x: fix_time(x))

In [44]:
orders_df['lead_time_good'].value_counts()

## We can see the groupings for the leadtimes (In a way to bin the lead times)

0:02:30    4450
0:12:00    4052
2:00:00    3351
3:00:00    2772
0:01:30    2227
0:01:00    2067
0:02:00     929
0:00:20     698
1:12:00     454
0:00:30     392
0:00:15     352
0:00:25     222
0:00:45      62
0:00:00      60
0:00:10      21
0:00:55       5
0:03:00       4
2:00:30       1
1:12:30       1
0:18:00       1
Name: lead_time_good, dtype: int64

### basket_size

In [45]:
# Check for nulls
orders_df['basket_size'].isnull().sum()

0

In [46]:
import decimal

In [47]:
def check_decimals(basket):
    d = decimal.Decimal(str(basket)) ##Cast back into string
    dp = d.as_tuple().exponent
    #print(dp)
    if dp < -2:
        print(dp)
        return False
    else:
        return True


In [48]:
orders_df['basket_valid'] = orders_df['basket_size'].apply(lambda x: check_decimals(x))

In [49]:
orders_df['basket_valid'].value_counts()

True    22121
Name: basket_valid, dtype: int64

In [50]:
orders_df['basket_size'].max()

##Looks abit suspicious

99999.0

In [51]:
orders_df['basket_size'].min()

3.01

In [52]:
## Lets view the basket_size distribution
orders_df['basket_size'].sort_values(ascending=False)

4145     99999.00
4180     99999.00
4170     99999.00
4171     99999.00
4172     99999.00
4173     99999.00
4174     99999.00
4175     99999.00
4176     99999.00
4177     99999.00
4178     99999.00
4179     99999.00
4181     99999.00
4192     99999.00
4182     99999.00
4183     99999.00
4184     99999.00
4185     99999.00
4186     99999.00
4187     99999.00
4244     99999.00
4188     99999.00
4189     99999.00
4190     99999.00
4169     99999.00
4168     99999.00
4167     99999.00
4166     99999.00
4147     99999.00
4148     99999.00
           ...   
785          4.68
5021         4.65
10078        4.64
21360        4.60
9599         4.55
21181        4.47
4046         4.41
10866        4.34
9796         4.33
9650         4.27
12160        4.23
1255         4.21
4558         4.19
1519         4.05
14992        4.02
11083        3.95
15410        3.93
11079        3.93
18657        3.90
1658         3.87
15055        3.72
13289        3.68
4560         3.60
17137        3.54
617       

In [53]:
orders_df.groupby(by='basket_size').count()

,submitted_on,pickup,delivery_date,customer_address_postal,promotion_code,lead_time,customer_index,menu_id,merchant_type,organisation_name_provided,postal_valid,postal_good,promo_good,lead_time_good,basket_valid
basket_size,,,,,,,,,,,,,,,
3.01,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1
3.02,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1
3.12,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1
3.38,2,2,2,0,0,2,2,2,2,2,2,0,0,2,2
3.43,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1
3.54,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1
3.60,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1
3.68,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1
3.72,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1


In [54]:
orders_df.loc[orders_df['basket_size']== 99999.00]
## We will have to drop these entries, as they are invalid

orders_df.loc[orders_df['basket_size']== 99999.00].index

Int64Index([4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065,
            ...
            4245, 4246, 4247, 4248, 4249, 4250, 4251, 4252, 4253, 4254],
           dtype='int64', length=199)

### basket size imputation to consider later (Not run)

#### Option 1: Create a new feature that imputes the 99999.0 and replaces it with the median of the merchant's basket size

In [ ]:
## We use the median to impute because the mean may be skewed by the rare-occasion large basket.
## Taking the median is a safer approach

#### Option 2: Dropping 199 rows.

In [ ]:
## We can afford to drop the 199 rows of data as it is less than 1% of our dataset.
(199/22121)*100

In [ ]:
## Drop invalid basket sizes
orders_df.drop(orders_df.loc[orders_df['basket_size']== 99999.00].index, axis=0, inplace=True)

In [ ]:
orders_df['basket_size'].sort_values(ascending=False)

### customer_index

In [55]:
orders_df['customer_index'].isnull().sum()

## No nulls for customer_id
## Check for validity later

0

### menu_id & merchant_type

In [56]:
orders_df['menu_id'].isnull().sum()
## No nulls for menu id

0

In [57]:
## Convert Data type for menu_id to int64
orders_df['menu_id'] = orders_df['menu_id'].astype(int)

In [58]:
## Menu_id must match up with merchant_type
orders_df['menu_id'].value_counts()

5    5363
1    4507
2    4312
3    3354
4    2776
6    1566
8     141
7     102
Name: menu_id, dtype: int64

In [59]:
orders_df['menu_id'].unique()

## We have menus 1-8

array([1, 2, 3, 4, 5, 6, 7, 8])

In [60]:
orders_df['merchant_type'].isnull().sum()

0

In [61]:
orders_df['merchant_type'].count()

22121

In [62]:
orders_df['merchant_type'].value_counts()


Cakes             10637
Dim Sum            5504
Korean Chicken     4312
Salad              1668
Name: merchant_type, dtype: int64

In [63]:
## Check for ids for CAKES
orders_df.loc[orders_df['merchant_type']=='Cakes']['menu_id'].unique()

array([1, 3, 4])

In [64]:
## Check for ids for DIM SUM
orders_df.loc[orders_df['merchant_type']=='Dim Sum']['menu_id'].unique()

array([5, 8])

In [65]:
## Check for ids for KOREAN CHICKEN
orders_df.loc[orders_df['merchant_type']=='Korean Chicken']['menu_id'].unique()

array([2])

In [66]:
## Check for ids for SALAD
orders_df.loc[orders_df['merchant_type']=='Salad']['menu_id'].unique()

array([6, 7])

**Merchant Menu ids to their types**
<table style="height: 95px;" border="1" width="281">
<tbody>
<tr>
<td style="width: 173px;">Cakes</td>
<td style="width: 92px;">1 , 3 , 4</td>
</tr>
<tr>
<td style="width: 173px;">Dim Sum</td>
<td style="width: 92px;">5 , 8</td>
</tr>
<tr>
<td style="width: 173px;">Korean Chicken</td>
<td style="width: 92px;">2</td>
</tr>
<tr>
<td style="width: 173px;">Salad</td>
<td style="width: 92px;">6 , 7</td>
</tr>
</tbody>
</table>

In [67]:
## Check for match up for amount of merchants_types to the ids

## The total amount of menu_ids of 1, 3 and 4 must == merchant_type Cakes count of 10637
orders_df.loc[(orders_df['menu_id'].isin([1,3,4]))]['merchant_type'].count()

10637

In [68]:
## example: The total amount of menu_ids of 5 and 8 must == merchant_type Dim Sum count of 5504
orders_df.loc[(orders_df['menu_id'].isin([5,8]))]['merchant_type'].count()

5504

In [69]:
## example: The total amount of menu_ids of 2 must == merchant_type Korean Chicken count of 4312
orders_df.loc[(orders_df['menu_id'].isin([2]))]['merchant_type'].count()

4312

In [70]:
## example: The total amount of menu_ids of 2 must == merchant_type Salad count of 1668
orders_df.loc[(orders_df['menu_id'].isin([6,7]))]['merchant_type'].count()

1668

In [71]:
orders_df.loc[(orders_df['menu_id'].isin([6,7])) & (orders_df['basket_size'] != 99999.0)]['basket_size'].sum()

214306.02

In [72]:
orders_df['promo_good'].notnull()

0        False
1        False
2        False
3        False
4         True
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12        True
13       False
14        True
15        True
16       False
17        True
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27        True
28       False
29       False
         ...  
22091    False
22092    False
22093    False
22094    False
22095     True
22096     True
22097    False
22098    False
22099    False
22100    False
22101    False
22102    False
22103     True
22104    False
22105    False
22106    False
22107    False
22108    False
22109    False
22110    False
22111     True
22112     True
22113     True
22114    False
22115    False
22116    False
22117    False
22118    False
22119    False
22120    False
Name: promo_good, Length: 22121, dtype: bool

In [73]:
orders_df.loc[(orders_df['menu_id'].isin([5])) & (orders_df['basket_size'] != 99999.0) & (orders_df['promo_good'].notnull())]['basket_size'].sum()

220188.31

### organisation_name_provided

In [74]:
orders_df['organisation_name_provided'].isnull().sum()

## has 7 nulls.

7

In [75]:
orders_df[orders_df['organisation_name_provided'].isnull()]

,submitted_on,pickup,delivery_date,customer_address_postal,promotion_code,lead_time,basket_size,customer_index,menu_id,merchant_type,organisation_name_provided,postal_valid,postal_good,promo_good,lead_time_good,basket_valid
3903,2018-06-11 08:43:15,1,2018-06-13 04:00:00,NaN,NaN,720.0,67.32,29703,1,Cakes,NaN,False,NaN,NaN,0:12:00,True
6741,2017-08-10 02:35:17,0,2017-08-12 08:30:00,731899,NaN,2880.0,143.20,23043,3,Cakes,NaN,True,731899,NaN,2:00:00,True
10327,2018-06-04 06:14:26,1,2018-06-09 04:00:00,NaN,NaN,720.0,138.26,32980,1,Cakes,NaN,False,NaN,NaN,0:12:00,True
13446,2018-05-25 06:00:35,1,2018-05-26 04:00:00,NaN,NaN,720.0,95.59,32980,1,Cakes,NaN,False,NaN,NaN,0:12:00,True
15275,2017-08-02 00:15:11,0,2017-08-07 10:30:00,520142,NaN,2880.0,76.80,25752,3,Cakes,NaN,True,520142,NaN,2:00:00,True
19595,2018-08-09 05:20:52,1,2018-08-25 06:00:00,NaN,NaN,4320.0,48.19,49359,4,Cakes,NaN,False,NaN,NaN,3:00:00,True
21142,2018-09-11 02:47:36,0,2018-09-15 04:00:00,178897,NaN,720.0,77.99,32844,1,Cakes,NaN,True,178897,NaN,0:12:00,True


In [76]:
## Since organisation_name_provided is a binary. We will impute all NaNs with 0

x = orders_df[orders_df['organisation_name_provided'].isnull()]

orders_df.loc[x.index, 'organisation_name_provided'] = 0

In [77]:
## Recheck for the Nulls
orders_df['organisation_name_provided'].isnull().sum()

0

In [78]:
orders_df['organisation_name_provided'] = orders_df['organisation_name_provided'].astype(int)

In [79]:
orders_df['organisation_name_provided'].value_counts()
## We have 4001 orders for organizations

0    18120
1     4001
Name: organisation_name_provided, dtype: int64

<hr>

In [80]:
#Check data types
orders_df.dtypes

submitted_on                  datetime64[ns]
pickup                                 int64
delivery_date                 datetime64[ns]
customer_address_postal               object
promotion_code                        object
lead_time                            float64
basket_size                          float64
customer_index                         int64
menu_id                                int64
merchant_type                         object
organisation_name_provided             int64
postal_valid                            bool
postal_good                           object
promo_good                            object
lead_time_good                        object
basket_valid                            bool
dtype: object

## Export  'cleaned' Orders dataset

In [81]:
orders_df.to_csv('orders_cleaned.csv')
## Exported out for Viz and further use

# Summary of Orders dataset

- The dataset consist of **22121 observations (rows) 11 Features (cols)**
- customer_address_postal has 3371 nulls
- promotion_code has 19237 nulls
- organisation_Name_provided has 7 nulls
- This dataset spans from **2016-10-01** to **2018-09-30**

<b>From the data dictionary provided in the Assignment PDF</b>
- submitted_on == Date Time for order submission
- pickup == 1/0 binary
- delivery_date == Date Time
- customer_address_postal   == String -> need to convert to 6 digit integer
- promotion_code == String obj
- lead_time == integer (No. of mins) -> Might convert to time object
- basket_size  == float ($$ worth of basket)
- customer_index == customer ID, unique integer (Match from customer table)
- menu_id == Merchant ID
- merchant_type == String obj categorical
- organisation_name_provided == binary 1/0 (Corporate or not)

<b>Data validity:</b>
- customer_address_postal: had to check for valid postal codes. Did a clean up to extract 6 digit postal codes. Those invalid was replaced with nulls.
    - postal_good is the new feature to hold the valid postal codes
- promotion_codes: has to do corrections on the strings, this allowed for the correct groupings of the promo codes
    - promo_good is the new feature to hold the corrected promo codes
- lead_time: was in minutes. Was reformated to 'DAYS:HRS:MINS' format, allowed for binning of the lead times
    - lead_time_good is the new feature to hold the reformatted lead times
- basket_size: **has 199 entries of 99999.0** >> These entries are would be deemed problematic, however we can revisit this to either impute or drop.
    - If to be imputed: We would use the median to impute because the mean may be skewed by the rare-occasion large basket. Taking the median is a safer approach.
    - For now, we will keep it and **EXCLUDE IT* for the Analysis (see tableau viz)
- menu_id and merchant_type: No issues here as they were checked to match up. (no clashing or errors)
- organisation_name_provided: nulls were imputed to 0

# Please see Tableau Dashboard for Orders for Dashboard Viz

https://public.tableau.com/profile/ziig.yee#!/vizhome/Oddle_Orders_Overview/SalesPerformanceDash

- Please view in fullscreen >> Click the bottom left of the viz
<img src=fullscreen.png>